# Binary Paint Shop Problem (BPSP): Example Usage

This notebook demonstrates how to solve a single Binary Paint Shop Problem (BPSP)
instance using **all algorithms provided in this repository**.

We consider:
- Classical heuristics:
  - Greedy
  - Red-First
  - Recursive Greedy
  - Recursive Star Greedy
- Quantum methods:
  - Recursive QAOA (RQAOA)
  - Expressive QAOA (XQAOA)

The goal is to minimize the number of **paint color swaps** in the sequence.

In [1]:
import numpy as np
from functools import partial
from scipy import optimize

# Core BPSP utilities
from BPSP import map_ising, get_edges_adj_mat, get_pos, get_swaps

# Classical heuristics
from BPSP_Greedy import greedy_solver, red_first_solver
from BPSP_Recursive import recursive_greedy
from BPSP_Star import recursive_star_greedy

# Quantum-inspired solvers
from RQAOA import GraphManager, RQAOA
from XQAOA import XQAOA, get_color_swaps, get_color_swaps_and_icc

## Loading a BPSP instance

We load a single instance with **128 cars** from the dataset.
Each instance consists of a sequence of length `2n`, where each car appears exactly twice.

In [3]:
# Problem size and instance ID
N = 128
INSTANCE_ID = 2

DATA_PATH = f"../bpsp_data/bpsp#{N}_{INSTANCE_ID}.txt"

with open(DATA_PATH, "r") as f:
    seq = list(map(int, f.read().split()))

car_pos = get_pos(np.array(seq))

bpsp_graph = map_ising(seq)
edges, adj_mat = get_edges_adj_mat(bpsp_graph)

print(f"Loaded instance n={N}, id={INSTANCE_ID}")
print(f"Sequence length = {len(seq)}")
print(f"Graph: |V|={adj_mat.shape[0]}, |E|={len(edges)}")

Loaded instance n=128, id=2
Sequence length = 256
Graph: |V|=128, |E|=248


### Greedy heuristic

The greedy solver assigns the first car a color and propagates colors forward
based on local consistency.

In [4]:
paint_seq, swaps = greedy_solver(seq, car_pos)
print(f"Greedy swaps: {swaps}  (ratio={swaps/N:.6f})")

Greedy swaps: 64  (ratio=0.500000)


### Red-First heuristic

The red-first heuristic assigns the first occurrence of every car to red (1)
and the second to blue (0).

In [5]:
paint_seq, swaps = red_first_solver(seq)
print(f"Red-First swaps: {swaps}  (ratio={swaps/N:.6f})")

Red-First swaps: 93  (ratio=0.726562)


### Recursive Greedy heuristic

This algorithm recursively removes cars and reinserts them while enforcing
local consistency rules.

In [6]:
paint_seq, swaps = recursive_greedy(seq, car_pos)
print(f"Recursive Greedy swaps: {swaps}  (ratio={swaps/N:.6f})")

Recursive Greedy swaps: 50  (ratio=0.390625)


### Recursive Star Greedy heuristic

An extension of Recursive Greedy that allows temporary “star” assignments
to defer decisions until more context is available.

In [7]:
paint_seq, swaps = recursive_star_greedy(seq, car_pos)
print(f"Recursive Star Greedy swaps: {swaps}  (ratio={swaps/N:.6f})")

Recursive Star Greedy swaps: 46  (ratio=0.359375)


## Recursive QAOA (RQAOA)

RQAOA repeatedly applies p=1 QAOA to identify strongly correlated variables,
eliminates them, and finally brute-forces the small residual instance.

In [8]:
# Choose eliminations: leaving 8 variables is your usual default
elim_steps = max(N - 8, 0)

gm = GraphManager(bpsp_graph, verbose=False)
best_cost, assignment = RQAOA(gm, elim_steps)

# Convert assignment dict -> colors and swaps using your helper
paint_seq, swaps = get_swaps(list(assignment.values()), car_pos)

print(f"RQAOA swaps: {swaps}  (ratio={swaps/N:.6f})")
print(f"RQAOA best_cost (Ising objective): {best_cost}")

RQAOA swaps: 40  (ratio=0.312500)
RQAOA best_cost (Ising objective): -176.0


## Expressive QAOA (XQAOA)

XQAOA generalizes QAOA by assigning **independent angles per node and per edge**.
We optimize these angles using classical optimizers and decode the result
into a paint sequence.

In [9]:
xqaoa_loss = partial(XQAOA, edges, adj_mat)
num_params = adj_mat.shape[0] + len(edges)

np.random.seed(40)
init_angles = np.random.uniform(0, 2*np.pi, num_params)

res = optimize.minimize(
    xqaoa_loss,
    init_angles,
    method="L-BFGS-B",
    options={"disp": False, "maxfun": 200000}
)

paint_seq, swaps = get_color_swaps(edges, adj_mat, res.x, car_pos)
print(f"XQAOA (1 run) swaps: {swaps}  (ratio={swaps/N:.6f})")

XQAOA (1 run) swaps: 40  (ratio=0.312500)
